In [14]:
import random
import numpy as np
import pandas as pd
import os

music_data_path = 'D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/labels.csv'
text_data_path = 'D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/Text_dataset/text_VA_clean.csv'

In [16]:
df = pd.read_csv(music_data_path)
print(df.columns)
del_VA_list = ["0491", "0838", "0907", "1511", "1595", "1752", "1803", "1809","1834","2110","2449","2726","2731","2815","2933"]
music_df = df[['Valence_Mean', 'Arousal_Mean']]
print(music_df)

Index(['Unnamed: 0', 'Valence', 'Arousal', 'Valence.1', 'Arousal.1',
       'Valence.2', 'Arousal.2', 'Valence.3', 'Arousal.3', 'Valence.4',
       'Arousal.4', 'Valence.5', 'Arousal.5', 'Valence_Mean', 'Arousal_Mean'],
      dtype='object')
      Valence_Mean  Arousal_Mean
0         7.333333      7.500000
1         4.166667      4.833333
2         7.333333      7.666667
3         4.166667      6.500000
4         7.166667      8.500000
...            ...           ...
2995      4.166667      4.000000
2996      3.666667      4.666667
2997      5.166667      5.833333
2998      3.333333      6.333333
2999      5.333333      7.000000

[3000 rows x 2 columns]


In [19]:
music_df.reset_index(drop=True, inplace=True)

# 要删除的索引列表（基于位置）
del_VA_list = [0, 2, 4, 6, 8]  # 示例索引位置

# 删除指定的行
music_df = music_df.drop(index=del_VA_list)
print(music_df.head)

<bound method NDFrame.head of       Valence_Mean  Arousal_Mean
1         4.166667      6.500000
3         7.166667      8.000000
5         7.000000      4.333333
7         3.833333      3.666667
9         3.500000      4.166667
...            ...           ...
2990      4.166667      4.000000
2991      3.666667      4.666667
2992      5.166667      5.833333
2993      3.333333      6.333333
2994      5.333333      7.000000

[2990 rows x 2 columns]>


In [12]:
print(f"Reading text data from: {text_data_path}")
dftext = pd.read_csv(text_data_path)
print(dftext.columns) 
text_df = dftext[['Valence','Arousal']]
print(text_df)

Reading text data from: D:/PolyU/URIS/Part2_projects/WEMOM_V1/Data/Text_dataset/text_VA_clean.csv
Index(['sentence', 'Valence', 'Arousal'], dtype='object')
       Valence  Arousal
0          5.3      4.6
1          4.7      5.0
2          6.6      4.6
3          6.9      5.7
4          7.1      5.8
...        ...      ...
12827      4.7      0.6
12828      9.3      6.2
12829      4.7      0.0
12830      4.0      5.0
12831      4.7      1.2

[12832 rows x 2 columns]


In [20]:
# 创建音乐数据集的字典
music_dataset = [
    {
        "index": i,
        "valence": row['Valence_Mean'],
        "arousal": row['Arousal_Mean']
    }
    for i, row in music_df.iterrows()
]

# 创建文本数据集的字典
text_dataset = [
    {
        "index": i,
        "valence": row['Valence'],  # 这里假设 V 是 Valence
        "arousal": row['Arousal']   # 这里假设 A 是 Arousal
    }
    for i, row in text_df.iterrows()
]

# 输出结果以验证
print("Music Dataset:", music_dataset[:5])  # 打印前5个样本
print("Text Dataset:", text_dataset[:5])    # 打印前5个样本

Music Dataset: [{'index': 1, 'valence': np.float64(4.166666667), 'arousal': np.float64(6.5)}, {'index': 3, 'valence': np.float64(7.166666667), 'arousal': np.float64(8.0)}, {'index': 5, 'valence': np.float64(7.0), 'arousal': np.float64(4.333333333)}, {'index': 7, 'valence': np.float64(3.833333333), 'arousal': np.float64(3.666666667)}, {'index': 9, 'valence': np.float64(3.5), 'arousal': np.float64(4.166666667)}]
Text Dataset: [{'index': 0, 'valence': np.float64(5.300000000000001), 'arousal': np.float64(4.6000000000000005)}, {'index': 1, 'valence': np.float64(4.699999999999999), 'arousal': np.float64(5.0)}, {'index': 2, 'valence': np.float64(6.6000000000000005), 'arousal': np.float64(4.6000000000000005)}, {'index': 3, 'valence': np.float64(6.9), 'arousal': np.float64(5.699999999999999)}, {'index': 4, 'valence': np.float64(7.1), 'arousal': np.float64(5.8)}]


In [21]:
import random

# 参数
n = 4  # 每个锚点的正负样本数量
num_anchors = 100  # 锚点数量
train_ratio = 0.8  # 训练集比例

# 计算情感相似度
def calculate_similarity(valence1, arousal1, valence2, arousal2):
    return ((valence1 - valence2) ** 2 + (arousal1 - arousal2) ** 2) ** 0.5

# 构建正负样本集
def build_samples(text_dataset, music_dataset, n, num_anchors):
    anchors = random.sample(text_dataset, num_anchors)  
    text_pos_samples = []
    text_neg_samples = []
    music_pos_samples = []
    music_neg_samples = []

    for anchor in anchors:
        anchor_index = anchor["index"]
        anchor_valence = anchor["valence"]
        anchor_arousal = anchor["arousal"]

        # 计算与音乐样本的相似度
        music_similarities = [
            {
                "index": music["index"],
                "similarity": calculate_similarity(
                    anchor_valence, anchor_arousal, music["valence"], music["arousal"]
                ),
            }
            for music in music_dataset
        ]
        music_similarities.sort(key=lambda x: x["similarity"]) 

        # 计算与文本样本的相似度
        text_similarities = [
            {
                "index": text["index"],
                "similarity": calculate_similarity(
                    anchor_valence, anchor_arousal, text["valence"], text["arousal"]
                ),
            }
            for text in text_dataset
        ]
        text_similarities.sort(key=lambda x: x["similarity"])  

        # 选取正负样本，并将主样本索引添加到每一行的开头
        music_pos_samples.append([anchor_index] + [x["index"] for x in music_similarities[:n]])
        music_neg_samples.append([anchor_index] + [x["index"] for x in music_similarities[-n:]])
        text_pos_samples.append([anchor_index] + [x["index"] for x in text_similarities[:n]])
        text_neg_samples.append([anchor_index] + [x["index"] for x in text_similarities[-n:]])

    return anchors, text_pos_samples, text_neg_samples, music_pos_samples, music_neg_samples

# 构建样本集
anchors, text_pos_samples, text_neg_samples, music_pos_samples, music_neg_samples = build_samples(
    text_dataset, music_dataset, n, num_anchors
)

# 分割为训练集和测试集
train_size = int(len(text_pos_samples) * train_ratio)

train_text_pos_samples = text_pos_samples[:train_size]
test_text_pos_samples = text_pos_samples[train_size:]

train_text_neg_samples = text_neg_samples[:train_size]
test_text_neg_samples = text_neg_samples[train_size:]

train_music_pos_samples = music_pos_samples[:train_size]
test_music_pos_samples = music_pos_samples[train_size:]

train_music_neg_samples = music_neg_samples[:train_size]
test_music_neg_samples = music_neg_samples[train_size:]

# 保存样本集索引
with open("train_text_pos_samples.txt", "w") as f:
    for samples in train_text_pos_samples:
        f.write(" ".join(map(str, samples)) + "\n")

with open("test_text_pos_samples.txt", "w") as f:
    for samples in test_text_pos_samples:
        f.write(" ".join(map(str, samples)) + "\n")

with open("train_text_neg_samples.txt", "w") as f:
    for samples in train_text_neg_samples:
        f.write(" ".join(map(str, samples)) + "\n")

with open("test_text_neg_samples.txt", "w") as f:
    for samples in test_text_neg_samples:
        f.write(" ".join(map(str, samples)) + "\n")

with open("train_music_pos_samples.txt", "w") as f:
    for samples in train_music_pos_samples:
        f.write(" ".join(map(str, samples)) + "\n")

with open("test_music_pos_samples.txt", "w") as f:
    for samples in test_music_pos_samples:
        f.write(" ".join(map(str, samples)) + "\n")

with open("train_music_neg_samples.txt", "w") as f:
    for samples in train_music_neg_samples:
        f.write(" ".join(map(str, samples)) + "\n")

with open("test_music_neg_samples.txt", "w") as f:
    for samples in test_music_neg_samples:
        f.write(" ".join(map(str, samples)) + "\n")

print("Samples saved!")

Samples saved!
